# Imports

In [10]:
#export
import os
import sys
sys.path.append(os.path.join(os.getcwd(),'exp'))

import fire
import docker
from config import cfg

# Code

In [48]:
#export
def status():
    pattern = cfg.DOCKER.CONTAINER_PREFIX
    print(f'\nLooking for *{pattern}* containers: ')
    
    for c in cycle_c_gen():
        gpu = check_gpu(c)
        print(f'\t{c.name} @ GPU{gpu} @ {c.status}')
    
def cycle_c_gen(pat=cfg.DOCKER.CONTAINER_PREFIX, list_all=False):
    client = docker.from_env()
    containers = client.containers.list(all=list_all)
    for i, c in enumerate(containers):
        print(f'{i}. Inspecting {c.name}')
        if pat in c.name:
            yield c
        else:
            print(f'\t Skipping {c.name}, not part of dHPO\n')
    
def check_gpu(container):
    env_vars = container.attrs['Config']['Env']
    g = None
    for v in env_vars:
        if v.startswith('CUDA_VISIBLE_DEVICES'):
            g = v.split("=")[-1]
    return g.split(',')

In [40]:
#export
def check_status(c):
    c.reload()
    return c.status

def is_running(c): return check_status(c) == 'running'
def is_paused(c): return check_status(c) == 'paused'

def unpause(c):
    if is_paused(c):
        print(f'\t UNpausing {c.name}...')
        c.unpause()
        print('\tSUCCESS')
        return True
    elif is_running(c):
        print(f'\tWARNING {c.name} already running!')
        return True
    else:
        print(f'\tSomething is wrong with {c.name}, check it manually')
        return False

def pause(c):
    if is_running(c):
        print(f'\t Pausing {c.name}...')
        c.pause()
        print('\tSUCCESS')
        return True
    elif is_paused(c):
        print(f'\tWARNING {c.name} already on pause!')
        return True
    else:
        print(f'\tSomething is wrong with {c.name}, check it manually')
        return False
    
def switch(gpus=None, mode=None):
    usage =''' Usage:
        switch $GPUS $MODE
        switch 0 pause
        switch 0,1 unpause
    '''
    if gpus is None or mode not in ['pause', 'unpause']:
        print(usage)
        return
        #exit(0)
    
    if not isinstance(gpus, tuple):
        gpus = gpus,
    gpus = set([str(i) for i in gpus])
    #print(gpus)
    if mode == 'pause':
        do = pause
    elif mode == 'unpause':
        do = unpause

    for c in cycle_c_gen():
        c_gpus = set(check_gpu(c))
        if c_gpus.intersection(gpus):
            print(f'\t{c.name} is using GPUS{c_gpus}, trying to set on "{mode}"')
            do(c)
        else:
            print(f'\tSkipping {c.name}, on GPU{c_gpus}')
            
def clean(force):
    if force == 'dhpo':
        for c in cycle_c_gen(list_all=True):
            if c.status == 'exited':
                c.remove()
    else:
        print('Usage: clean dhpo')

In [ ]:
#export
if __name__ == '__main__':
    fire.Fire({'status':status, 'switch':switch, 'clean':clean})

# Tests 

In [44]:
switch(gpus=1, mode='unpause')


0. Inspecting sokolov_cycle_May_15_20_16_05_358866
	Skipping sokolov_cycle_May_15_20_16_05_358866, on GPU{'2'}

1. Inspecting sokolov_cycle_May_15_20_16_05_341391
	sokolov_cycle_May_15_20_16_05_341391 is using GPUS{'1'}, trying to set on "unpause"
	 UNpausing sokolov_cycle_May_15_20_16_05_341391...
	SUCCESS

2. Inspecting sokolov_cycle_May_15_20_16_05_314989
	Skipping sokolov_cycle_May_15_20_16_05_314989, on GPU{'2'}

3. Inspecting sokolov_cycle_May_15_20_16_05_304330
	Skipping sokolov_cycle_May_15_20_16_05_304330, on GPU{'0'}

4. Inspecting sokolov_cycle_May_15_20_16_05_303681
	sokolov_cycle_May_15_20_16_05_303681 is using GPUS{'1'}, trying to set on "unpause"
	 UNpausing sokolov_cycle_May_15_20_16_05_303681...
	SUCCESS

5. Inspecting sokolov_cycle_May_15_20_16_05_271560
	Skipping sokolov_cycle_May_15_20_16_05_271560, on GPU{'0'}

6. Inspecting sokolov_post
	 Skipping sokolov_post, not part of dHPO



In [28]:
client = docker.from_env()
containers = client.containers.list(all=True)
c = containers[-2]
c.name, c.status

('recursing_herschel', 'exited')

# Exports

In [54]:
!python3 extra/n2s.py tools.ipynb

Converted tools.ipynb to exp/nb_tools.py


In [43]:
!python3 exp/nb_tools.py switch 0 unpause

0. Inspecting sokolov_dHPO_May_19_21_21_34_058049
	sokolov_dHPO_May_19_21_21_34_058049 is using GPUS{'0'}, trying to set on "unpause"
	 UNpausing sokolov_dHPO_May_19_21_21_34_058049...
	SUCCESS
1. Inspecting sokolov_post
	 Skipping sokolov_post, not part of dHPO



In [56]:
!python3 exp/nb_tools.py clean dhpo

0. Inspecting sokolov_dHPO_May_19_22_50_34_871651
1. Inspecting sokolov_dHPO_May_19_22_50_34_871256
2. Inspecting sokolov_dHPO_May_19_22_50_34_861781
3. Inspecting sokolov_dHPO_May_19_22_50_34_827169
4. Inspecting sokolov_dHPO_May_19_22_41_44_477678
5. Inspecting sokolov_dHPO_May_19_22_41_44_444603
6. Inspecting sokolov_dHPO_May_19_22_41_44_385144
7. Inspecting sokolov_dHPO_May_19_22_41_44_339823
8. Inspecting sokolov_dHPO_May_19_22_39_36_155521
9. Inspecting sokolov_dHPO_May_19_22_39_36_109078
10. Inspecting sokolov_dHPO_May_19_22_37_18_051127
11. Inspecting sokolov_dHPO_May_19_22_37_17_932803
12. Inspecting recursing_herschel
	 Skipping recursing_herschel, not part of dHPO

13. Inspecting sokolov_post
	 Skipping sokolov_post, not part of dHPO

